## ARCH
----

AutoRegressive Conditional Heteroscedasticit - пытаемся объяснить дисперсию в ряде через предыдущие значения (применяя к ним AR)


Пусть временной ряд представляется в таком виде:

$\epsilon_t = z_t * \sqrt{\omega + \sum_{k=1}^p \alpha_i \epsilon_{k-i}^2}$ при условии, что $\omega > 0, 1 \geq \alpha_i \geq 0$

где на этот раз:

$\epsilon_t$ - модель временного ряда, который мы пытаемся смоделировать
$z_t$ - некая константа представляющая непрогнозируемый шум и по условию $E[z_t] = 0, E[z^2_t] = 1$
$\omega$ - случайная ошибка (bias)
$\alpha_i$ - коэффициенты авторегрессии


Тогда условная дисперсия ряда будет равна

$\sigma_t^2 = E[\epsilon^2_t | \epsilon_{t-1}, ..., \epsilon_{t-q}) = \sigma^2_t = \omega + \sum_{k=1}^p \alpha_k \epsilon_{t-k}^2$

Получили модель ARCH(q) условной дисперсии. Требуем, чтобы все коэффициенты были больше 0 (иначе может получится отрицательная дисперсия)


Как помним из курса статистики:
1. Гетероскедастичность - дисперсия изменяется с течением времени.
2. Гомоскедастичность - дисперсия постоянна (выражается в виде шума, случаного блуждания).


Почему Conditional: вводят предположение, что дисперсия взаимозависима от ближайших значений ряда.

Тогда, имеет смысл попробовать построить модель, которая зависит от своих предыдущих значений во временном ряду.

Недостатки модели ARCH/GARCH:
* Не подходит для долгосрочного прогноза (как и впрочем почти все модели для временных рядов) :)

In [ ]:
import pandas as pd
import numpy as np
import hvplot
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import pacf, plot_pacf, acf, plot_acf
from statsmodels.tsa.stattools import adfuller

In [8]:
def tsplot(y, lags=None, figsize=(20, 7), style='bmh'):
    test_stationarity(y)
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):
        plt.figure(figsize=figsize)
        layout = (4, 1)
        ts_ax = plt.subplot2grid(layout, (0, 0), rowspan=2)
        acf_ax = plt.subplot2grid(layout, (2, 0))
        pacf_ax = plt.subplot2grid(layout, (3, 0))

        y.plot(ax=ts_ax, color='blue', label='Or')
        ts_ax.set_title('Original')
        plot_acf(y, lags=lags, ax=acf_ax, alpha=0.05)
        plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.05)
        plt.tight_layout()
        plt.show()
    return


def test_stationarity(timeseries):
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for [key, value] in dftest[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    print(dfoutput)

_Имитация ARCH(1) для временного ряда:_

In [ ]:
# уравнение вида: Var(yt) = a_0 + a_1*y{t-1}**2
# если a_1 между 0 and 1 тогда yt случайное блуждание
np.random.seed(13)


a0 = 2
a1 = .5

y = w = np.random.normal(size=100)
Y = np.empty_like(y)

for t in range(len(y)):
    Y[t] = w[t] * np.sqrt((a0 + a1*y[t-1]**2))

# simulated ARCH(1) series, looks like white noise
tsplot(Y)

## GARCH
-----

Как было сказано выше GARCH - дисперсия (волатильность) полностью зависит от случайного шума:

$\sigma_t^2 = E[\epsilon^2_t | \epsilon_{t-1}, ..., \epsilon_{t-q}) = \sigma^2_t = \omega + \sum_{k=1}^p \alpha_k \epsilon_{t-k}^2$

Мы хотим сделать дисперсию более или менее постоянной, для улучшения прогнозной способности модели. Для этого и необходим немного улучшенный подход, который получил название GARCH. Как выглядит модель GARCH:

$\sigma_t^2 =E[\epsilon^2_t | \epsilon_{t-1}, ..., \epsilon_{t-q}) = \omega + \sum_{k=1}^p \alpha_k \epsilon_{t-k}^2 + \sum_{k=1}^q \beta_k \sigma_{t-k}^2$


Таким образом мы просто добавляем зависимость от прошлых значений для условной дисперсиии. Получаем модель GARCH(p,q), то есть это по сутии таже самая ARMA примененная к дисперсии временного ряда.


In [4]:
# pip install arch
from arch import arch_model

Как было показано на предыдущих занятиях, моделирование и прогнозирование временного ряда состоит обычно из 3 шагов:
1. Создаем экземпляр модели - передаем данные и гиперпараметры.
2. Обучаем модель
3. Прогнозируем
<br>
</br>

```
1 from arch import arch_model
2
3 model = arch_model(
4               y=training_data,
5               x=None,          # внешние данные (exogenous data)
6               mean='Constant', # среднее значение модели, e.g. 'Zero', 'ARX'
7               vol='GARCH',     # тип модели (ARCH, GARCH, EGARCH и т.д.)
8               p=1,             # параметр p
9               q=1,             # игнорируется если модель типа ARCH
10              dist='Normal'    # тип распределения (StudentsT, Exp, Beta)
11 )
12 result = model.fit()
13 result.ssummary()
```

Прогнозирование при помощи GARCH реализовано иначе, нам необходимо спрогнозировать статистику $\epsilon_t$ (среднее и дисперсию).
Нас волнует теперь только дисперсия (или стандартное отклонение - так называемая волатильность). Таким образом мы хотим спрогнозировать параметр $\sigma_t$
<br>
</br>

```
1 forecast_result = result.forecast(
2                           horizon=h,
3                           reindex=False,
4                           start='2022-10-10')
5 result.conditional_volatility # обученные предиктор, то что нам необходимо
6 np.ssqrt(forecast_result.variance) # условная дисперсия нашего прогноза
```

__Аргумент reindex:__ Реиндексирует прогноз таким образом, чтобы размерности прогноза совпадали с размерностью временного ряда.
__Зачем:__ При помощи модели GARCH мы можешь реализовывать прогноз с любой точки временного ряда (например если данные с Jan 2021 - Jan 2022, мы можем выбрать точку в качестве прогноза Mar 2021 и тогда с нее начнется прогноз). По умолчанию прогноз начинается с последней точки данных временного ряда. Кроме того, мы можем также реализовывать прогноз не только начиная с какой-либо точки данных временного ряда, но и организовать прогноз для каждой точки временного ряда одновременно, например, если горизонт временного ряда 2021-2022, то у нас есть возможность организовать прогноз начиная с Jan-21, Feb-21, ..., Dec-22.
<br>
</br>
__reindex=True__: выводит датафрейм такой же размерности как и тренировочный временной ряд.
__reindex=False__: выводит датафрейм, который содержит только тот горизонт, для которого мы бы хотели сделать прогноз.

Предположим, что горизонт временного ряда March-1 - March-10:
```
forecast(horizon=5, reindex=True, start='March 6')
```
![](./../src/imgs/garch_fcast.png)


Расшифровка столбцов: Каждый столбец - это прогноз на N+1 горизонт, например в нашем случуае h.1 = March-7, h.2 = March-8, ..., h.5=March-10.

In [4]:
import pandas as pd
import numpy as np
from arch import arch_model
import hvplot
import tqdm
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import matplotlib.pyplot as plt
from scipy.stats import boxcox
from math import sqrt
# import warnings
# warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('./../data/SPY.csv', parse_dates=True, index_col='Date')
df.head()

In [60]:
df['LogReturns'] = np.log(df['Close']).diff()

In [62]:
#df2 = df.loc['2010-01-05':'2015-01-01'].copy()
df2 = df.iloc[1:].copy()

In [ ]:
hvplot.plot(df2['LogReturns'], kind='line', width=800, height=400, label='Logarithmic Returns')

In [63]:
df2['Ysq'] = df['LogReturns'] ** 2

In [ ]:
hvplot.plot(df2['Ysq'], kind='line', width=800, height=400, label='Scaled Logarithmic Returns')

_Ислледуем PACF/ACF:_

In [ ]:
plot_acf(df2['LogReturns'])

In [ ]:
plot_pacf(df2['LogReturns'])

In [ ]:
plot_acf(df2['Ysq'])

In [ ]:
plot_pacf(df2['Ysq'])

In [ ]:
noise_sq = np.random.randn(500)**2
plot_acf(noise_sq)

In [ ]:
plot_pacf(noise_sq)

_Train/Test Split_:

In [67]:
Ntest = 500
train = df2.iloc[:-Ntest][['LogReturns']].copy()
test = df2.iloc[-Ntest:][['LogReturns']].copy()

_Зачем производить шкалирование_:

In [68]:
model = arch_model(train['LogReturns'], vol='GARCH', p=1, q=1)

In [ ]:
res = model.fit()

_ARCH(1)_:

In [70]:
m = train['LogReturns'].mean()
s = test['LogReturns'].std()

train['Scaled'] = (train['LogReturns'] - m) / s
test['Scaled'] = (test['LogReturns'] - m) / s
df2['Scaled'] = (df2['LogReturns'] - m) / s

In [71]:
train['Scaled'] = train.Scaled.astype(np.float)
test['Scaled'] = train.Scaled.astype(np.float)
df2['Scaled'] = train.Scaled.astype(np.float)

In [72]:
arch_one = arch_model(train['Scaled'], vol='GARCH', p=1)

In [ ]:
res_arch_one = arch_one.fit(update_freq=10)

In [ ]:
res_arch_one.summary()

In [ ]:
df2['ARCH_1'] = res_arch_one.conditional_volatility
hvplot.plot(df2[['Scaled', 'ARCH_1']], kind='line', width=800, height=400, label='Scaled Log Returns and Conditional Volatility')

In [81]:
res_arch_one.forecast(horizon=Ntest)

In [82]:
forecast_arch_one = res_arch_one.forecast(horizon=Ntest, reindex=True)

In [83]:
forecast_arch_one

_Разница между reindex=False и reindex=True_

In [ ]:
forecast_arch_one.mean

In [ ]:
forecast_arch_one.variance

In [ ]:
forecast_arch_one.residual_variance

In [87]:
forecast_arch_one = res_arch_one.forecast(horizon=Ntest, reindex=False)

In [ ]:
forecast_arch_one.mean

In [ ]:
forecast_arch_one.variance

In [ ]:
forecast_arch_one.residual_variance

Прогнозирование

In [ ]:
df2.loc['2011-08-09':]['ARCH_1'].plot(figsize=(15,5))

In [90]:
fcast_arch1 = res_arch_one.forecast(horizon=Ntest, reindex=True, start='2011-08-01')

In [ ]:
fcast_arch1.variance['2011-07-28':'2011-08-05']

In [ ]:
fcast_arch1.variance.loc['2011-08-09'].to_numpy().shape

In [ ]:
df2.index.get_loc('2011-08-09')

In [94]:
df2.index[402+500]

Timestamp('2013-08-06 00:00:00')

In [96]:
df2.loc['2011-08-10':'2013-08-06', 'ARCH_1 Forecast'] = np.sqrt(
    fcast_arch1.variance.loc['2011-08-09'].to_numpy()
)

In [ ]:
df2.loc['2011-08-01':'2012-02-01'][['ARCH_1','ARCH_1 Forecast']].plot(figsize=(15,5))
plt.show()

Выберем другую дату

In [ ]:
fcast_arch1.variance.index.get_loc('2011-08-17')

In [ ]:
df2.index[408+500]

In [101]:
df2.loc['2011-08-18':'2013-08-14', 'ARCH_1 Forecast Low'] = np.sqrt(
    fcast_arch1.variance.loc['2011-08-17'].to_numpy()
)

In [ ]:
plot_cols = ['ARCH_1 Forecast', 'ARCH_1 Forecast Low']
df2.loc['2011-08-01':'2012-02-01'][plot_cols].plot(figsize=(15,5))
plt.show()

GARCH(1, 1)

In [103]:
garch11 = arch_model(train['Scaled'], vol='GARCH', p=1, q=1)

In [ ]:
res_garch11 = garch11.fit(update_freq=10)

In [ ]:
res_garch11.summary()

In [ ]:
df2['GARCH(1,1)'] = res_garch11.conditional_volatility
df2[['Scaled', 'ARCH_1', 'GARCH(1,1)']].plot(figsize=(15,5))
plt.show()

In [108]:
fcast_garch11 = res_garch11.forecast(
    horizon=Ntest, reindex=False, start='2011-08-09'
)

In [109]:
df2.loc['2011-08-10':'2013-08-06', 'GARCH(1,1) Forecast'] = np.sqrt(
    fcast_garch11.variance.loc['2011-08-09'].to_numpy()
)

In [110]:
df2['AbsScaled'] = df2['Scaled'].abs()

In [ ]:
plot_cols = ['AbsScaled', 'ARCH_1 Forecast', 'GARCH(1,1) Forecast']
df2.loc['2011-08-01':'2012-02-01'][plot_cols].plot(figsize=(15,5))
plt.show()

# Практика: создайте модель GARCH (p, q) с параметрами на выбор. Распределение выбрать StudentsT

In [114]:
garchpq = arch_model(train['Scaled'], p=8, q=5, dist='StudentsT')

In [115]:
res_garchpq = garchpq.fit(update_freq=10)

Iteration:     10,   Func. Count:    185,   Neg. LLF: 2678.256488785693
Iteration:     20,   Func. Count:    368,   Neg. LLF: 2525.536748176307
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2525.2882244412085
            Iterations: 29
            Function evaluations: 520
            Gradient evaluations: 29


In [ ]:
res_garchpq.summary()

In [ ]:
res_arch_one.aic, res_garchpq.aic, res_garch11.aic

In [ ]:
df2['GARCH(p,q)']=res_garchpq.conditional_volatility
df2[['Scaled', 'GARCH(p,q)']].plot(figsize=(15,5))
plt.show()

In [119]:
fcast_garchpq = res_garchpq.forecast(horizon=Ntest, reindex=False, start='2011-08-09')

In [120]:
df2.loc['2011-08-10':'2013-08-06', 'GARCH(p,q) Forecast'] = np.sqrt(fcast_garchpq.variance.loc['2011-08-09'].to_numpy())

In [ ]:
plot_cols = ['AbsScaled', 'GARCH(p,q) Forecast']
df2.loc['2011-08-01':'2012-02-01'][plot_cols].plot(figsize=(15,5))
plt.show()

Как еще можно использовать GARCH в связке с ARIMA:

In [ ]:
series = pd.read_csv('./../data/air_passengers.csv')['passengers']
series = boxcox(series, 0)
series = series[12:] - series[:-12]
series = series[1:] - series[:-1]
tsplot(series)
plt.show()

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
mdl = ARIMA(series, order=(4,0,4)).fit()
tsplot(mdl.resid)

In [ ]:
from statsmodels.graphics.tsaplots import plot_predict
with plt.style.context('bmh'):
    plt.figure(figsize=(14,8))
    # ax = plt.axes()
    plot_predict(mdl)
    plt.plot(series, color='red', label='Series')
    plt.legend()
    plt.show()

In [ ]:
def _get_best_model(TS):
    best_aic = np.inf
    best_order = None
    best_mdl = None

    for i in range(5):
        for d in range(5):
            for j in range(5):
                try:
                    tmp_mdl = ARIMA(TS, order=(i,d,j)).fit(method='statespace')
                    tmp_aic = tmp_mdl.aic
                    if tmp_aic < best_aic:
                        best_aic = tmp_aic
                        best_order = (i, d, j)
                        best_mdl = tmp_mdl
                except: continue
    print('aic: {:6.5f} | order: {}'.format(best_aic, best_order))
    return best_aic, best_order, best_mdl

aic, order, mdl = _get_best_model(series)

In [ ]:
tsplot(mdl.resid)

Применим GARCH

In [ ]:
# Now we can fit the arch model using the best fit arima model parameters
p_ = order[0]
o_ = order[1]
q_ = order[2]

# Using student T distribution usually provides better fit
am = arch_model(series, p=p_, o=o_, q=q_, dist='StudentsT')
res = am.fit(update_freq=5, disp='off')
print(res.summary())

In [ ]:
tsplot(res.resid)

# Полезные материалы:

1. [Статистическое обоснование модели GARCH](https://online.stat.psu.edu/stat510/lesson/11/11.1)
2. [Use Case ARCH/GARCH](https://medium.com/@ranjithkumar.rocking/time-series-model-s-arch-and-garch-2781a982b448)
3. [Vector Autoregressive Model](https://kevinkotze.github.io/ts-7-var/)
4. [VARIMA](https://analyticsindiamag.com/a-guide-to-varma-with-auto-arima-in-time-series-modelling/)
5. [VARIMA/VARMA](https://faculty.washington.edu/ezivot/econ584/notes/varModels.pdf)